# Movie Recommendation Based on Content-based

Importing librarues

In [68]:
pip install pandas scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [134]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns


Calling the dataset

In [149]:
df = pd.read_csv('tmdb_5000_movies.csv')

In [150]:
print(df)



         budget                                             genres  \
0     237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1     300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2     245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3     250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4     260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
...         ...                                                ...   
4798     220000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4799       9000  [{"id": 35, "name": "Comedy"}, {"id": 10749, "...   
4800          0  [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...   
4801          0                                                 []   
4802          0                [{"id": 99, "name": "Documentary"}]   

                                               homepage      id  \
0                           http://www.avatarmovie.com/   19995   
1          http://disney.

In [151]:
print(df.columns) 

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')


In [152]:
df = df[['original_title','overview']]


In [209]:
df

,original_title,overview
0,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,Spectre,A cryptic message from Bond’s past sends him o...
3,The Dark Knight Rises,Following the death of District Attorney Harve...
4,John Carter,"John Carter is a war-weary, former military ca..."
...,...,...
4798,El Mariachi,El Mariachi just wants to play his guitar and ...
4799,Newlyweds,A newlywed couple's honeymoon is upended by th...
4800,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic..."
4801,Shanghai Calling,When ambitious New York attorney Sam is sent t...


In [204]:
print(df.head(10)) 



                             original_title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   
5                              Spider-Man 3   
6                                   Tangled   
7                   Avengers: Age of Ultron   
8    Harry Potter and the Half-Blood Prince   
9        Batman v Superman: Dawn of Justice   

                                            overview  
0  In the 22nd century, a paraplegic Marine is di...  
1  Captain Barbossa, long believed to be dead, ha...  
2  A cryptic message from Bond’s past sends him o...  
3  Following the death of District Attorney Harve...  
4  John Carter is a war-weary, former military ca...  
5  The seemingly invincible Spider-Man goes up ag...  
6  When the kingdom's most wanted-and most charmi...  
7  When Tony Stark tries to jumpstart a do

 Checking if there are  missing values and removing them. 

In [154]:
print(df.isnull().sum()) 


original_title    0
overview          3
dtype: int64


In [155]:
df.dropna(inplace=True)



Transforming the overview column in the vector form to compute similarity

In [156]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [157]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')

In [158]:
matrix = tf.fit_transform(df['overview'])



computing cosine similarity to check  all movies are of the same content on the basis of the overview column that was present in the data set

In [159]:
from sklearn.metrics.pairwise import linear_kernel



In [160]:
cosine_similarities = linear_kernel(matrix,matrix)



 reseting the index with the movie name that is the original title and defining a function for the recommendation that will search for similar movies by checking cosine similarities 

In [161]:
movie_title = df['original_title']



In [198]:
indices = pd.Series(df.index, index=df['original_title'])

def movie_recommend(original_title):

    idx = indices[original_title]

    sim_scores = list(enumerate(cosine_similarities[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]

    return movie_title.iloc[movie_indices]

Top 5 recommendations for 2 different movies 

In [220]:
 movie_recommend('Harry Potter and the Half-Blood Prince').head(5)




114           Harry Potter and the Goblet of Fire
113     Harry Potter and the Order of the Phoenix
4133                                        Plush
191      Harry Potter and the Prisoner of Azkaban
1205                             The Skeleton Key
Name: original_title, dtype: object

In [221]:
movie_recommend('Bring It On').head(5)



3169    The Purge: Anarchy
2078            Swing Vote
1407          House of Wax
2537           Wag the Dog
3733          Ночной дозор
Name: original_title, dtype: object